## Identify candidate pairs to test

In this step, we would first identify the candidate paralog pairs to test. For the candidate pairs, we hvae A1 and A2 two genes. A2 is the gene with poorly represented HD and A1 is A2's paralog gene(s).

**Criteria for the candidate pairs:**
- A2 is homozygously deleted in 1 or 2 cell lines (we already have the poorly represented genes) **(filter1)**
- A1 has its gene dependency score in CRISPR gene effect dataset **(filter 2)**
- The minimum sequence identity between A1 and A2 is at least 20% (and they are already listed in De Kegel's gene list) **(filter 3)**
- Closest paralog pairs **(filter 3)**
- Both genes in paralog pairs are broadly expressed **(filter 4)**
    - RNA tissue distribution = 'Detected in all' OR
    - RNA tissue distribution = 'Detected in many' AND RNA tissue specificity = Low tissue specificity', 'Not detected'
- Paralog genes in a gene pair should not be in the same chromosomal location **(filter 5)**


**Input**
- Paralog gene list (~36.6k paralog pairs from Ensembl): DeKegel_TableS8.csv (https://doi.org/10.1101/2020.12.16.423022). In paralog gene list, Barbara had already annotated the minimum sequence identity and paralog gene family size.
- Protein atlas tissue distribution + specificity: proteinatlas.tsv (https://v19.proteinatlas.org/about/download).
- HGNC information: hgnc_gene_with_protein_product.txt (https://www.genenames.org/download/statistics-and-files/).
- Sanger CNV data (annotated cell lines with BROAD ID and filtered based on paralog gene list): CNV_sanger_paralog.csv
- BROAD CRISPR gene effect data (annotated cell lines with BROAD ID and filtered based on paralog gene list): CRISPR_broad_paralog.csv
- 1942 genes with poorly represented HD: poorly_represented_HD_gene.csv


**Output**
- Candidate pairs list (341 sorted gene pairs and 374 pairs to test): pairs_to_test.csv

In [25]:
## Import modules
import pandas as pd

In [26]:
## Read the paralog information data
paralog_pairs = pd.read_csv('/Users/amy/Desktop/SyntheticLethalityProject/sources/DeKegel_TableS8.csv', index_col = False, usecols = [2,3,4,5,6,7,8,14,15,17])
paralog_pairs[:2]

,sorted_gene_pair,A1,A2,A1_entrez,A2_entrez,A1_ensembl,A2_ensembl,min_sequence_identity,closest,family_size
0,SMARCA2_SMARCA4,SMARCA2,SMARCA4,6595,6597,ENSG00000080503,ENSG00000127616,0.746812,True,2
1,EXOC6_EXOC6B,EXOC6,EXOC6B,54536,23233,ENSG00000138190,ENSG00000144036,0.698159,True,2


**Filter 1**: A2 is homozygously deleted in 1 or 2 cell lines

In [27]:
# Read the pre-saved poorly represented HD list
poor_repr_ls = pd.read_csv('/Users/amy/Desktop/SyntheticLethalityProject/1_data_processing/05_poorly_represented_HD_in_CCLs/poorly_represented_HD_gene.csv', index_col = False)
poor_repr_ls[:2]

,entrez_id,hgnc_symbol,cell_line_frequency,cell_line_frequency_category
0,9,NAT1,1,1
1,10,NAT2,1,1


In [28]:
# Identify the paralog pairs with at least one gene lost (i.e., homozygous deletion).
poor_paralog_pairs_filterA1 = paralog_pairs[paralog_pairs.A1_entrez.isin(poor_repr_ls['entrez_id'])]
poor_paralog_pairs_filterA2 = paralog_pairs[paralog_pairs.A2_entrez.isin(poor_repr_ls['entrez_id'])]
poor_paralog_pairs_filter = pd.concat([poor_paralog_pairs_filterA1, poor_paralog_pairs_filterA2])
poor_paralog_pairs_filter = poor_paralog_pairs_filter.drop_duplicates(subset=['sorted_gene_pair'])


# Change the column order of the dataframe so that we have two combinations for one sorted_gene_pair
# Example, sorted gene pair: CNOT7(A1)_CNOT8(A2)
# There can be two combinations: CNOT7(A1) and CNOT8(A2); CNOT7(A2) and CNOT8(A1)
pre_candidate_pairs = poor_paralog_pairs_filter[['sorted_gene_pair', 'A2', 'A1', 'A2_entrez', 'A1_entrez', 'A2_ensembl', 'A1_ensembl']]
pre_candidate_pairs = pre_candidate_pairs.rename(columns={'A2':'A1', 'A1':'A2', 
                                                          'A2_entrez':'A1_entrez', 
                                                          'A1_entrez':'A2_entrez', 
                                                          'A2_ensembl': 'A1_ensembl', 
                                                          'A1_ensembl':'A2_ensembl'})

# Combine two dataset. Hence, the candidate pairs contain all the possibilities of the combinations for a specific sorted_gene_pair
pre_candidate_pairs_full = pd.concat([poor_paralog_pairs_filter, pre_candidate_pairs])

# Merge the pre_candidate_pairs_full data with the rare_homodel data
poor_repr_ls = poor_repr_ls.rename(columns={'entrez_id':'A2_entrez'})
pairs_to_test_filter1 = pd.merge(poor_repr_ls, pre_candidate_pairs_full, on = ['A2_entrez'], how = 'left')
pairs_to_test_filter1 = pairs_to_test_filter1[['sorted_gene_pair', 
                                               'A1', 'A2', 
                                               'A1_entrez', 'A2_entrez', 
                                               'A1_ensembl','A2_ensembl',
                                               'hgnc_symbol']]
pairs_to_test_filter1 = pairs_to_test_filter1.rename(columns={'hgnc_symbol':'A2_hgnc_symbol'})


print("Number of candidate pairs (A2 lost in 1 or 2 cell lines):", pairs_to_test_filter1.sorted_gene_pair.unique().shape[0])

Number of candidate pairs (A2 lost in 1 or 2 cell lines): 10015


**Filter 2**: A1 has available gene dependency scores

In [29]:
crispr_broad = pd.read_csv('/Users/amy/Desktop/SyntheticLethalityProject/1_data_processing/04_paralog_genes/CRISPR_broad_paralog.csv', index_col=None)
CNV_sanger = pd.read_csv('/Users/amy/Desktop/SyntheticLethalityProject/1_data_processing/04_paralog_genes/CNV_sanger_paralog.csv', index_col = None)
pairs_to_test_filter2 = pairs_to_test_filter1[(pairs_to_test_filter1['A1_entrez'].astype(str).isin(crispr_broad.columns))]

print("Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score):", pairs_to_test_filter2.sorted_gene_pair.unique().shape[0])

Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score): 9277


**Filter 3**: Paralog gene pairs with min. sequence identity should be higher than 20%, and closest 

In [30]:
# paralog_pairs_filter3 = paralog_pairs[(paralog_pairs.family_size <= 20) & (paralog_pairs.min_sequence_identity >= 0.2) & (paralog_pairs.closest == True)]
paralog_pairs_filter3 = paralog_pairs[(paralog_pairs.min_sequence_identity >= 0.2) & (paralog_pairs.closest == True)]
pairs_to_test_filter3 = pairs_to_test_filter2[pairs_to_test_filter2['sorted_gene_pair'].astype(str).isin(paralog_pairs_filter3['sorted_gene_pair'])]
print('Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score; min_seq_id >= 0.2; closest):', pairs_to_test_filter3.sorted_gene_pair.unique().shape[0])

Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score; min_seq_id >= 0.2; closest): 1029


**Filter 4**: Both genes are broadly expressed across tissues instead of expressing in a tissue specific manner

In [31]:
## Load protein expression data
protein_atlas = pd.read_csv('/Users/amy/Desktop/SyntheticLethalityProject/sources/proteinatlas.tsv', sep='\t', index_col = False)
protein_atlas = protein_atlas[['Ensembl', 'Gene', 'RNA tissue specificity', 'RNA tissue distribution']]

# Genes that are broadly expressed across tissues 
protein_atlas_filter = protein_atlas[protein_atlas['RNA tissue distribution'].isin(['Detected in all']) |
                                     (protein_atlas['RNA tissue distribution'].isin(['Detected in many']) &
                                      protein_atlas['RNA tissue specificity'].isin(['Low tissue specificity','Not detected']))]

# Filter the paralog genes that are broadly expressed across tissues 
# The final data are the candidate paralog pairs list 
pairs_to_test_filter4 = pairs_to_test_filter3[pairs_to_test_filter3.A1_ensembl.isin(protein_atlas_filter.Ensembl) & pairs_to_test_filter3.A2_ensembl.isin(protein_atlas_filter.Ensembl)]

print('Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score; min_seq_id >= 0.2; closest; both genes are broadly expressed):', 
      pairs_to_test_filter4.sorted_gene_pair.unique().shape[0])

Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score; min_seq_id >= 0.2; closest; both genes are broadly expressed): 370


**Filter 5**: Both genes in a paralog pairs should not be in the same chromosomal location (because they might be co-deleted

In [32]:
## First, find the gene pairs with paralog genes locate within the same chromosomal region 
# Read HGNC data 
hgnc = pd.read_table('/Users/amy/Desktop/SyntheticLethalityProject/sources/hgnc_gene_with_protein_product.txt', index_col = None, low_memory=False)
# Subset the data
hgnc = hgnc[['symbol', 'location']]

# Subset the pairs_to_test_filter4 data (only get the useful column)
# First, add two column. Add a column for candidate pairs to test (for both entrez ID and gene symbol)
pd.DataFrame(pairs_to_test_filter4).loc[:,'pairs_to_test'] = pairs_to_test_filter4['A1_entrez'].astype(str).str.cat(pairs_to_test_filter4['A2_entrez'].astype(str), sep='-')
pd.DataFrame(pairs_to_test_filter4).loc[:,'pairs_to_test_symbol'] = pairs_to_test_filter4['A1'].astype(str).str.cat(pairs_to_test_filter4['A2'].astype(str), sep='_')

pairs_A1 = pairs_to_test_filter4[['A1', 'pairs_to_test_symbol']]
pairs_A2 = pairs_to_test_filter4[['A2', 'pairs_to_test_symbol']]

# Map the chromosomal location to the A1 and A2 gene 
hgnc_map = dict(zip(hgnc['symbol'].astype(str), hgnc['location'].astype(str)))
pd.DataFrame(pairs_A1).loc[:, 'chr_loc_A1'] = pairs_A1['A1'].map(hgnc_map)
pd.DataFrame(pairs_A2).loc[:, 'chr_loc_A2'] = pairs_A2['A2'].map(hgnc_map)

# Merge the two separated dataframe together 
pairs_chr_loc = pd.DataFrame(pd.merge(pairs_A1, pairs_A2, on = ['pairs_to_test_symbol'], how = 'left'))

# check if chr_loc_A1 and chr_loc_A2 has the same location for A1 and A2
pairs_chr_loc['chr_loc_same'] = pairs_chr_loc['chr_loc_A1'] == pairs_chr_loc['chr_loc_A2']
# List of pairs that locate within the same chromosomal location 
pairs_same_chr_loc = pairs_chr_loc[pairs_chr_loc['chr_loc_same'] == True].pairs_to_test_symbol

## Filter
pairs_to_test_filter5 = pairs_to_test_filter4[~(pairs_to_test_filter4['pairs_to_test_symbol'].isin(list(pairs_same_chr_loc)))]

In [33]:
## How many pairs to test?
print('Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score; min_seq_id >= 0.2; closest; both genes are broadly expressed; different chromosomal locations):', 
      pairs_to_test_filter5.sorted_gene_pair.unique().shape[0])

print('Number of pairs to test (after all 5 filtering mentioned above):', pairs_to_test_filter5.pairs_to_test_symbol.shape[0])

Number of candidate pairs (A2 lost in 1 or 2 cell lines; available A1 dependency score; min_seq_id >= 0.2; closest; both genes are broadly expressed; different chromosomal locations): 341
Number of pairs to test (after all 5 filtering mentioned above): 374


In [34]:
## Save the data 
pairs_to_test_filter5.to_csv('/Users/amy/Desktop/SyntheticLethalityProject/2_data_analysis/01_candidate_pairs_to_test/pairs_to_test.csv', index = False)